In [ ]:
import numpy as np
import pandas as pd

# Dữ liệu huấn luyện
# Tạo DataFrame chứa dữ liệu bệnh nhân
# Your code here

data = pd.DataFrame({
    "Sốt": ["Cao ", "Cao", "Cao ", "Thấp", " Cao ", "Thấp", "Thấp", " Cao ", "Thấp", "Cao"],
    "Ho": ["Có", "Không", "Có", "Có", "Có", "Không", "Có", "Không", "Không", "Có"],
    "Đau họng": ["Có", "Có", "Không", "Có", "Có", "Có", "Không", "Có", "Không", "Có"],
    "Mệt mỏi": ["Có", "Không", "Có", "Có", "Không", "Không", "Có", "Có", "Không", "Có"],
    "Flu": ["Có", "Có", "Có", "Không", "Có", "Không", "Không", "Có", "Không", "Có"]
})

# Dữ liệu bệnh nhân cần dự đoán
new_patient = {"Sốt": "Cao", "Ho": "Có", "Đau họng": "Có", "Mệt mỏi": "Không"}

# Tính xác suất tiên nghiệm P(C)
P_flu = data["Flu"].value_counts(normalize = True)["Có"] # P(C =1)
P_not_flu = 1 - P_flu # P(C=0)

# Hàm tính xác suất có điều kiện P(x_i |C) với Laplace smoothing
def conditional_prob(feature, value, flu_status):
    # Laplace smoothing thêm 1 vào tử số để tránh xác suất = 0
    # Ví dụ: Nếu không có trường hợp nào "Sốt=Cao" khi "Flu=Có" thì P(Sốt=Cao|Flu=Có) = 0
    # Điều này sẽ làm cho tích các xác suất = 0, gây mất thông tin
    count_feature = data[(data[feature] == value) & (data["Flu"] == flu_status)].shape[0]

    # Thêm 2 vào mẫu số vì mỗi đặc trưng có 2 giá trị (VD: Sốt: Cao/Thấp)
    # Để đảm bảo tổng xác suất = 1: P(Cao|Flu) + P(Thấp|Flu) = 1
    count_flu = data[data["Flu"] == flu_status].shape[0]
    prob = (count_feature + 1) / (count_flu + 2)

    print(f"P({feature}={value}|Flu={flu_status}) = {prob:.4f}")
    
    return prob

# Tính P(X|C) = P(x_1 |C) * P(x_2 |C) * ... * P( x_n |C) theo Naive Bayes
P_X_given_flu = 1 # P(X|C=1)
P_X_given_not_flu = 1 # P(X|C =0)

for feature, value in new_patient.items():
    P_X_given_flu *= conditional_prob(feature, value, "Có")
    # print(f"P_X_given_flu: {P_X_given_flu}")
    P_X_given_not_flu *= conditional_prob(feature, value, "Không")
    # print(f"P_X_given_not_flu: {P_X_given_not_flu}")

# Tính P(C|X) trực tiếp từ tỷ lệ(không cần P(X))
numerator_flu = P_X_given_flu * P_flu
numerator_not_flu = P_X_given_not_flu * P_not_flu

# Chu ẩn hóa xác suất
total = numerator_flu + numerator_not_flu
P_flu_given_X = numerator_flu/ total
P_not_flu_given_X = numerator_not_flu/ total

print("\nKết quả Naive Bayes:")
print(f"P(Flu = Có|X) = {P_flu_given_X:.4f}")
print(f"P(Flu = Không|X) = {P_not_flu_given_X:.4f}")

# Kết luận
print("\nKết luận:", "Bệnh nhân có khả năng bị cúm." if P_flu_given_X > P_not_flu_given_X 
else "Bệnh nhân KHÔNG có khả năng bị cúm.")


P(Sốt=Cao|Flu=Có) = 0.3750
P(Sốt=Cao|Flu=Không) = 0.1667
P(Ho=Có|Flu=Có) = 0.6250
P(Ho=Có|Flu=Không) = 0.5000
P(Đau họng=Có|Flu=Có) = 0.7500
P(Đau họng=Có|Flu=Không) = 0.5000
P(Mệt mỏi=Không|Flu=Có) = 0.3750
P(Mệt mỏi=Không|Flu=Không) = 0.5000

Kết quả Naive Bayes:
P(Flu = Có|X) = 0.8260
P(Flu = Không|X) = 0.1740

Kết luận: Bệnh nhân có khả năng bị cúm.


In [2]:
data.shape[0]

10

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$


$$P(Y|X) = \frac{P(X|Y)P(Y)}{P(X)}$$


$$P(A \cap B) = P(A|B)P(B) = P(B|A)P(A)$$
